In [40]:
import pandas as pd
import numpy as np

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from haversine import haversine

In [234]:
cols = {
        'infer' : [
            'FROM',
            'TO',
            'hour',
            'weekday',
            'week',
            'year',
            'passenger_count',
            'dist_e',
            'dist_t',
            'pickup_longitude',
            'dropoff_longitude',
            'pickup_latitude',
            'dropoff_latitude'
        ],
        
        'train' : [
            'fare_amount',
            'FROM',
            'TO',
            'hour',
            'weekday',
            'week',
            'year',
            'passenger_count',
            'pickup_longitude',
            'dropoff_longitude',
            'pickup_latitude',
            'dropoff_latitude'
        ]
    }

new_cols =  {
    'infer' : [
        'from',
        'to',
        'hour',
        'weekday',
        'week',
        'year',
        'p_count',
        'dist_e',
        'dist_t',
        'x0',
        'x1',
        'y0',
        'y1'
    ],
    
    'train' : [
        'fare',
        'from',
        'to',
        'hour',
        'weekday',
        'week',
        'year',
        'p_count',
        'dist_e',
        'dist_t',
        'x0',
        'x1',
        'y0',
        'y1'
    ]
}


def process_datetime(df):
    df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    df['hour'] = df['pickup_datetime'].dt.hour
    df['weekday'] = df['pickup_datetime'].dt.dayofweek
    df['week'] = df['pickup_datetime'].dt.weekofyear
    df['year'] = df['pickup_datetime'].dt.year
    df.drop('pickup_datetime', axis=1, inplace=True)
    
def geo_location(row):
    locations = {
    'Manhattan' :  [(-74.0200424, 40.7127848),(-74.0291405, 40.6853894),(-74.0237331, 40.6820700),(-74.0146351, 40.6853243),
                    (-74.0115452, 40.6892294),(-74.0081120, 40.6942405),(-74.0010738, 40.7026348),(-73.9930916, 40.7062135),
                    (-73.9854527, 40.7071895),(-73.9769554, 40.7081004),(-73.9701748, 40.7162979),(-73.9701748, 40.7319743),
                    (-73.9701748, 40.7406891),(-73.9651966, 40.7432253),(-73.9584160, 40.7501180),(-73.9510345, 40.7573351),
                    (-73.9424300, 40.7678504),(-73.9396405, 40.7710031),(-73.9393830, 40.7739769),(-73.9382887, 40.7757643),
                    (-73.9358425, 40.7781367),(-73.9330101, 40.7789166),(-73.9290619, 40.7794366),(-73.9258432, 40.7806064),
                    (-73.9209080, 40.7834335),(-73.9123249, 40.7928888),(-73.9137197, 40.7956829),(-73.9156508, 40.7973236),
                    (-73.9169222, 40.7980383),(-73.9183545, 40.7986191),(-73.9195830, 40.7991348),(-73.9203340, 40.7995328),
                    (-73.9212459, 40.8008322),(-73.9219058, 40.8016931),(-73.9233649, 40.8021032),(-73.9249635, 40.8022372),
                    (-73.9270878, 40.8021560),(-73.9281499, 40.8032524),(-73.9295769, 40.8046249),(-73.9322484, 40.8072236),
                    (-73.9330745, 40.8099196),(-73.9332676, 40.8131759),(-73.9329028, 40.8187785),(-73.9347267, 40.8332294),
                    (-73.9289761, 40.8448528),(-73.9246416, 40.8511831),(-73.9160585, 40.8615700),(-73.9103508, 40.8688723),
                    (-73.9094925, 40.8720527),(-73.9124966, 40.8742918),(-73.9178610, 40.8752653),(-73.9238262, 40.8780559),
                    (-73.9327955, 40.8803921),(-73.9441681, 40.8567987),(-73.9527512, 40.8413465),(-73.9621925, 40.8249814),
                    (-73.9963531, 40.7771617),(-74.0112877, 40.7570100),(-74.0203857, 40.7129149)],
    
    'JFK':         [(-73.8124180, 40.6606849),(-73.8186407, 40.6614988),(-73.8219881, 40.6602617),(-73.8237906, 40.6552481),
                    (-73.8220310, 40.6489967),(-73.8196707, 40.6462942),(-73.7852097, 40.6293273),(-73.7907887, 40.6237901),
                    (-73.7842655, 40.6185131),(-73.7774849, 40.6258748),(-73.7736225, 40.6224872),(-73.7723351, 40.6183177),
                    (-73.7671852, 40.6215752),(-73.7670135, 40.6250930),(-73.7658978, 40.6275685),(-73.7482166, 40.6342126),
                    (-73.7466502, 40.6398303),(-73.7482381, 40.6435915),(-73.7545681, 40.6474338),(-73.7546968, 40.6491595),
                    (-73.7666273, 40.6556062),(-73.7725496, 40.6588618),(-73.7791157, 40.6617267),(-73.7842226, 40.6631265),
                    (-73.7894154, 40.6639729),(-73.7949514, 40.6647541),(-73.8004017, 40.6651773),(-73.8041353, 40.6651122),
                    (-73.8077402, 40.6644937),(-73.8097572, 40.6638101),(-73.8113451, 40.6629637),(-73.8123322, 40.6606849)],
    
    'LaGardia':    [(-73.8893652, 40.7737006),(-73.8884640, 40.7691587),(-73.8880348, 40.7687361),(-73.8875198, 40.7671029),
                    (-73.8852024, 40.7677367),(-73.8812542, 40.7696543),(-73.8783574, 40.7711656),(-73.8765764, 40.7716369),
                    (-73.8743448, 40.7719294),(-73.8727355, 40.7720106),(-73.8714480, 40.7717669),(-73.8696241, 40.7711981),
                    (-73.8679075, 40.7704831),(-73.8632727, 40.7669079),(-73.8613629, 40.7667454),(-73.8587236, 40.7669566),
                    (-73.8564491, 40.7680780),(-73.8549685, 40.7695893),(-73.8539815, 40.7715069),(-73.8541102, 40.7725631),
                    (-73.8713837, 40.7807689),(-73.8706970, 40.7816463),(-73.8686800, 40.7838235),(-73.8692379, 40.7847983),
                    (-73.8687658, 40.7857406),(-73.8709116, 40.7868129),(-73.8751602, 40.7819063),(-73.8790226, 40.7836285),
                    (-73.8801813, 40.7819713),(-73.8792157, 40.7816219),(-73.8789797, 40.7808258),(-73.8855028, 40.7802490),
                    (-73.8846231, 40.7755206),(-73.8850951, 40.7754068),(-73.8848591, 40.7742369),(-73.8893652, 40.7737331)],
    
    'Newark':      [(-74.1883993, 40.7081004),(-74.1963816, 40.6929715),(-74.1928196, 40.6911167),(-74.1928089, 40.6900185),
                    (-74.1959310, 40.6892375),(-74.1956949, 40.6879440),(-74.1973042, 40.6880253),(-74.1987097, 40.6860240),
                    (-74.1974866, 40.6853488),(-74.1952658, 40.6806543),(-74.1935492, 40.6797918),(-74.1911674, 40.6759514),
                    (-74.1870904, 40.6732336),(-74.1800308, 40.6704995),(-74.1773701, 40.6695067),(-74.1518998, 40.7058557),
                    (-74.1550756, 40.7074010),(-74.1609335, 40.7096456),(-74.1688943, 40.7108005),(-74.1759753, 40.7106703),
                    (-74.1884208, 40.7080679)],
    
    'Teterboro':   [(-74.0792441, 40.8445119),(-74.0766478, 40.8435867),(-74.0763044, 40.8429536),(-74.0714121, 40.8402426),
                    (-74.0634942, 40.8387978),(-74.0607905, 40.8367848),(-74.0532589, 40.8434568),(-74.0514994, 40.8480667),
                    (-74.0535164, 40.8528711),(-74.0516281, 40.8573830),(-74.0571642, 40.8607911),(-74.0629148, 40.8660489),
                    (-74.0794373, 40.8447229)],
    
    'Brooklin':    [(-74.0049362, 40.5659810),(-73.9012527, 40.5761518),(-73.8936996, 40.5719793),(-73.8717270, 40.5784986),
                    (-73.8818550, 40.6070456),(-73.8933563, 40.6159071),(-73.8883781, 40.6243765),(-73.8763618, 40.6312815),
                    (-73.8622856, 40.6379252),(-73.8422012, 40.6432658),(-73.8364506, 40.6454475),(-73.8415146, 40.6658609),
                    (-73.8445187, 40.6806706),(-73.8461494, 40.6847386),(-73.8506985, 40.6906937),(-73.8525009, 40.6943381),
                    (-73.8528872, 40.6981775),(-73.8718128, 40.6946960),(-73.8688946, 40.6913119),(-73.8877773, 40.6832416),
                    (-73.8888073, 40.6849338),(-73.8924980, 40.6837623),(-73.8946009, 40.6850965),(-73.8985062, 40.6802149),
                    (-73.9050293, 40.6834368),(-73.9051151, 40.6866260),(-73.8996649, 40.6931342),(-73.8979053, 40.6969736),
                    (-73.8929701, 40.7015937),(-73.8898373, 40.7081980),(-73.8992357, 40.7123619),(-73.9025831, 40.7137932),
                    (-73.9059734, 40.7145739),(-73.9078617, 40.7158262),(-73.9101148, 40.7182170),(-73.9134407, 40.7224128),
                    (-73.9150500, 40.7244943),(-73.9176679, 40.7257790),(-73.9218950, 40.7264294),(-73.9249849, 40.7274214),
                    (-73.9280319, 40.7282019),(-73.9356279, 40.7291450),(-73.9389324, 40.7304459),(-73.9399195, 40.7327873),
                    (-73.9419365, 40.7356490),(-73.9474297, 40.7376000),(-73.9524078, 40.7390308),(-73.9561415, 40.7393885),
                    (-73.9594889, 40.7379577),(-73.9615917, 40.7357791),(-73.9630508, 40.7340881),(-73.9654541, 40.7215346),
                    (-73.9685440, 40.7157774),(-73.9709902, 40.7103125),(-73.9750242, 40.7081980),(-73.9781141, 40.7063437),
                    (-73.9872551, 40.7056930),(-73.9930058, 40.7053351),(-73.9997864, 40.7024396),(-74.0086699, 40.6875698),
                    (-74.0241194, 40.6782947),(-74.0181541, 40.6616290),(-74.0377235, 40.6426146),(-74.0439034, 40.6259399),
                    (-74.0409851, 40.6124213),(-74.0324450, 40.6040481),(-74.0075111, 40.5954458),(-74.0021896, 40.5868424),
                    (-74.0020180, 40.5837136),(-74.0090561, 40.5829314),(-74.0133476, 40.5796720),(-74.0147209, 40.5748479),
                    (-74.0054512, 40.5654594)],
    
    'Bronx':       [(-73.9239013, 40.8800839),(-73.9196527, 40.8764172),(-73.9182150, 40.8757358),(-73.9163053, 40.8757439),
                    (-73.9132798, 40.8749651),(-73.9111662, 40.8741052),(-73.9097071, 40.8731479),(-73.9089453, 40.8718985),
                    (-73.9098787, 40.8680854),(-73.9105868, 40.8666493),(-73.9116168, 40.8654241),(-73.9138377, 40.8629575),
                    (-73.9175177, 40.8588112),(-73.9209080, 40.8550135),(-73.9239120, 40.8505339),(-73.9265728, 40.8470928),
                    (-73.9285469, 40.8437814),(-73.9299631, 40.8406322),(-73.9318943, 40.8373205),(-73.9327526, 40.8356646),
                    (-73.9332676, 40.8325800),(-73.9329243, 40.8294952),(-73.9323664, 40.8230004),(-73.9321947, 40.8157905),
                    (-73.9325809, 40.8107560),(-73.9324951, 40.8091644),(-73.9291477, 40.8048766),(-73.9272380, 40.8032037),
                    (-73.9250708, 40.8023591),(-73.9232683, 40.8023428),(-73.9216805, 40.8015469),(-73.9209509, 40.8006535),
                    (-73.9204574, 40.7996302),(-73.9172602, 40.7981358),(-73.9116383, 40.7962677),(-73.9028835, 40.8036097),
                    (-73.8973045, 40.8052989),(-73.8917255, 40.8047792),(-73.8870049, 40.8025702),(-73.8849449, 40.8008160),
                    (-73.8784218, 40.8012058),(-73.8724136, 40.7999713),(-73.8668346, 40.8063384),(-73.8688517, 40.8096029),
                    (-73.8671565, 40.8100739),(-73.8602901, 40.8088883),(-73.8595819, 40.8059486),(-73.8556123, 40.8039508),
                    (-73.8476729, 40.8045356),(-73.8467932, 40.8063709),(-73.8486385, 40.8093918),(-73.8461494, 40.8102363),
                    (-73.8409138, 40.8180964),(-73.8407421, 40.8142639),(-73.8409996, 40.8118604),(-73.8386822, 40.8065333),
                    (-73.8322449, 40.8036747),(-73.8300133, 40.8043244),(-73.8313007, 40.8082224),(-73.8287258, 40.8101714),
                    (-73.8264084, 40.8104312),(-73.8153362, 40.8127699),(-73.8089848, 40.8113407),(-73.8046074, 40.8078327),
                    (-73.7968826, 40.8038047),(-73.7895012, 40.8026352),(-73.7879562, 40.8084823),(-73.7991142, 40.8116006),
                    (-73.7953377, 40.8154982),(-73.8027191, 40.8197853),(-73.8058090, 40.8261505),(-73.8114738, 40.8253711),
                    (-73.8138771, 40.8348529),(-73.8137054, 40.8416062),(-73.8106155, 40.8462812),(-73.8119888, 40.8506962),
                    (-73.8109589, 40.8578374),(-73.8078690, 40.8555004),(-73.8040924, 40.8488783),(-73.8011742, 40.8456319),
                    (-73.7960243, 40.8532931),(-73.7937927, 40.8495275),(-73.7896729, 40.8388790),(-73.7853813, 40.8356322),
                    (-73.7793732, 40.8361517),(-73.7771416, 40.8412166),(-73.7807465, 40.8513454),(-73.7843513, 40.8566689),
                    (-73.7840080, 40.8626411),(-73.7798882, 40.8739348),(-73.7821198, 40.8799054),(-73.9112091, 40.9148098),
                    (-73.9238691, 40.8802299)],
    
    'Staten':      [(-74.0817547, 40.6491921),(-74.0860248, 40.6491595),(-74.0899944, 40.6478245),(-74.0988350, 40.6453824),
                    (-74.1038561, 40.6461965),(-74.1077614, 40.6460011),(-74.1118813, 40.6455127),(-74.1189623, 40.6431681),
                    (-74.1236830, 40.6418005),(-74.1303778, 40.6419959),(-74.1369438, 40.6420610),(-74.1442823, 40.6402048),
                    (-74.1492605, 40.6394884),(-74.1527367, 40.6400746),(-74.1624355, 40.6410841),(-74.1651392, 40.6432333),
                    (-74.1720057, 40.6448289),(-74.1747093, 40.6454150),(-74.1774130, 40.6451870),(-74.1833782, 40.6452196),
                    (-74.1877127, 40.6439497),(-74.1911888, 40.6411818),(-74.1953516, 40.6375345),(-74.1986132, 40.6339847),
                    (-74.2017460, 40.6311186),(-74.2023039, 40.6230410),(-74.2012310, 40.6162654),(-74.1973257, 40.6154184),
                    (-74.1971970, 40.6128122),(-74.1986990, 40.6094892),(-74.1987419, 40.6062963),(-74.1974545, 40.6030706),
                    (-74.1980982, 40.5994213),(-74.1993427, 40.5960324),(-74.2053938, 40.5899384),(-74.2050934, 40.5854410),
                    (-74.2065954, 40.5805195),(-74.2097282, 40.5685891),(-74.2120457, 40.5633729),(-74.2124319, 40.5599822),
                    (-74.2189121, 40.5568847),(-74.2304134, 40.5568195),(-74.2363358, 40.5531676),(-74.2439747, 40.5482764),
                    (-74.2439747, 40.5482764),(-74.2492533, 40.5434826),(-74.2469358, 40.5388515),(-74.2456484, 40.5345789),
                    (-74.2437172, 40.5310889),(-74.2441893, 40.5265222),(-74.2440176, 40.5200957),(-74.2477942, 40.5192802),
                    (-74.2519999, 40.5162135),(-74.2547894, 40.5121026),(-74.2565918, 40.5079914),(-74.2565489, 40.5029663),
                    (-74.2538023, 40.4988872),(-74.2476225, 40.4945795),(-74.2399836, 40.4963744),(-74.2329025, 40.4996052),
                    (-74.2292547, 40.5003884),(-74.2235041, 40.5006168),(-74.2174101, 40.5020200),(-74.2117453, 40.5054137),
                    (-74.2081404, 40.5103080),(-74.2014027, 40.5115805),(-74.1947508, 40.5083177),(-74.1877127, 40.5152347),
                    (-74.1824341, 40.5189865),(-74.1779709, 40.5183993),(-74.1561699, 40.5277292),(-74.1425657, 40.5350356),
                    (-74.1374588, 40.5268811),(-74.1168594, 40.5447218),(-74.1063881, 40.5481133),(-74.0909386, 40.5641553),
                    (-74.0487099, 40.6003988),(-74.0638161, 40.6204676),(-74.0708971, 40.6251908),(-74.0696526, 40.6405305),
                    (-74.0694380, 40.6449917),(-74.0818405, 40.6492898)],
        
    'Queens':      [(-73.9410353, 40.5400093),(-73.8193703, 40.5794113),(-73.7898445, 40.5852780),(-73.7625504, 40.5884067),
                    (-73.7518644, 40.5893192),(-73.7458992, 40.5935232),(-73.7377453, 40.5937187),(-73.7374878, 40.5977594),
                    (-73.7380028, 40.6012786),(-73.7386465, 40.6043088),(-73.7425947, 40.6079905),(-73.7454271, 40.6119000),
                    (-73.7477016, 40.6110530),(-73.7479430, 40.6114602),(-73.7481093, 40.6118389),(-73.7549329, 40.6108575),
                    (-73.7641382, 40.6134312),(-73.7658978, 40.6155324),(-73.7669277, 40.6220963),(-73.7466717, 40.6320631),
                    (-73.7440538, 40.6373879),(-73.7430239, 40.6379578),(-73.7416935, 40.6378927),(-73.7413073, 40.6374205),
                    (-73.7412429, 40.6369971),(-73.7422299, 40.6361504),(-73.7426162, 40.6354013),(-73.7423372, 40.6349128),
                    (-73.7411571, 40.6348965),(-73.7402773, 40.6350105),(-73.7394619, 40.6350757),(-73.7406421, 40.6381044),
                    (-73.7422514, 40.6405630),(-73.7405777, 40.6474338),(-73.7360716, 40.6493549),(-73.7311792, 40.6500712),
                    (-73.7249565, 40.6519597),(-73.7256002, 40.6577224),(-73.7258148, 40.6600013),(-73.7280893, 40.6633544),
                    (-73.7291193, 40.6665445),(-73.7281752, 40.6702553),(-73.7262440, 40.6744867),(-73.7245703, 40.6819073),
                    (-73.7237120, 40.6855847),(-73.7255144, 40.6897826),(-73.7268877, 40.7084908),(-73.7277031, 40.7149317),
                    (-73.7305355, 40.7218273),(-73.7234974, 40.7246244),(-73.7136269, 40.7266408),(-73.7083054, 40.7276816),
                    (-73.7005806, 40.7388032),(-73.7015247, 40.7523287),(-73.7471867, 40.7802815),(-73.7506199, 40.7823612),
                    (-73.7796307, 40.8021804),(-73.7968826, 40.8005561),(-73.8137054, 40.8026352),(-73.8308716, 40.8021154),
                    (-73.8525009, 40.7987368),(-73.8652039, 40.7888599),(-73.8710833, 40.7866179),(-73.8730145, 40.7854807),
                    (-73.8787651, 40.7849608),(-73.8824558, 40.7854482),(-73.8858891, 40.7861955),(-73.8923264, 40.7878202),
                    (-73.8977337, 40.7905495),(-73.9098787, 40.7937986),(-73.9152002, 40.7878202),(-73.9167881, 40.7858706),
                    (-73.9195347, 40.7833360),(-73.9234400, 40.7803140),(-73.9249420, 40.7790466),(-73.9273882, 40.7779417),
                    (-73.9293623, 40.7766743),(-73.9306068, 40.7768693),(-73.9308643, 40.7779092),(-73.9328384, 40.7784616),
                    (-73.9346409, 40.7781692),(-73.9368725, 40.7777142),(-73.9379025, 40.7756018),(-73.9382458, 40.7735544),
                    (-73.9376020, 40.7724819),(-73.9365292, 40.7719944),(-73.9352417, 40.7717344),(-73.9361858, 40.7696543),
                    (-73.9377737, 40.7689068),(-73.9396191, 40.7682892),(-73.9415073, 40.7672166),(-73.9445114, 40.7635112),
                    (-73.9481592, 40.7587654),(-73.9547253, 40.7511583),(-73.9570427, 40.7492077),(-73.9588022, 40.7469643),
                    (-73.9609909, 40.7439406),(-73.9619780, 40.7420548),(-73.9628792, 40.7393560),(-73.9627934, 40.7381854),
                    (-73.9618063, 40.7379577),(-73.9593601, 40.7379903),(-73.9580297, 40.7388357),(-73.9555514, 40.7394372),
                    (-73.9535236, 40.7393885),(-73.9509380, 40.7388194),(-73.9496183, 40.7383317),(-73.9470220, 40.7376082),
                    (-73.9424944, 40.7359091),(-73.9410353, 40.7347060),(-73.9397907, 40.7317467),(-73.9379454, 40.7300231),
                    (-73.9343405, 40.7286572),(-73.9301348, 40.7281043),(-73.9288902, 40.7274214),(-73.9275169, 40.7260880),
                    (-73.9263582, 40.7229332),(-73.9246416, 40.7200384),(-73.9230108, 40.7167207),(-73.9233541, 40.7157124),
                    (-73.9245129, 40.7152895),(-73.9241266, 40.7140534),(-73.9198887, 40.7114511),(-73.9219058, 40.7093772),
                    (-73.9088488, 40.7016751),(-73.9109731, 40.6995766),(-73.9027441, 40.6948180),(-73.9042783, 40.6932480),
                    (-73.8927555, 40.6870166),(-73.8849449, 40.6866911),(-73.8691521, 40.6948912),(-73.8656330, 40.6807031),
                    (-73.8637877, 40.6810286),(-73.8610840, 40.6699949),(-73.8570499, 40.6699624),(-73.8557625, 40.6639078),
                    (-73.8609123, 40.6592199),(-73.8608265, 40.6557039),(-73.8574791, 40.6517969),(-73.8552475, 40.6431356),
                    (-73.8397980, 40.6415725),(-73.8401413, 40.6209888),(-73.8377380, 40.6139524),(-73.8284683, 40.6095217),
                    (-73.8298416, 40.5954458),(-73.8233185, 40.5910139),(-73.8428879, 40.5844958),(-73.8559341, 40.5826706),
                    (-73.8710403, 40.5738048),(-73.8940430, 40.5701538),(-73.9204788, 40.5662418),(-73.9393616, 40.5586780),
                    (-73.9489746, 40.5513742),(-73.9411211, 40.5400093),(-73.9410353, 40.5400093)],
        
    'Westchester': [(-73.6576653, 40.9848225),(-73.6604118, 40.9880620),(-73.6574936, 40.9900056),(-73.6598969, 40.9990752),
                    (-73.6550903, 41.0115114),(-73.7285614, 41.1004397),(-73.4848022, 41.2086221),(-73.5493469, 41.2881262),
                    (-73.5452271, 41.3757781),(-73.9800453, 41.3190756),(-73.9688444, 41.3079226),(-73.9651108, 41.3028290),
                    (-73.9536953, 41.2977350),(-73.9335251, 41.2901900),(-73.9498329, 41.2762584),(-73.9640808, 41.2646464),
                    (-73.9676857, 41.2501931),(-73.9494896, 41.2339294),(-73.9158440, 41.2170159),(-73.8918114, 41.1972565),
                    (-73.9072609, 41.1844678),(-73.8986778, 41.1643109),(-73.8816833, 41.1808504),(-73.8734436, 41.1605631),
                    (-73.8767052, 41.0829742),(-73.8696671, 41.0728809),(-73.8784218, 41.0304204),(-73.8887215, 40.9896169),
                    (-73.9045143, 40.9445094),(-73.9139557, 40.9172745),(-73.7690735, 40.8760116),(-73.7582588, 40.9018364),
                    (-73.7059021, 40.9345246),(-73.7009239, 40.9314121),(-73.6710548, 40.9526777),(-73.6497688, 40.9779541),
                    (-73.6576653, 40.9847253),(-73.6576653, 40.9848225)],
    
    'Nassau':      [(-73.7556839, 40.5800632),(-73.5774994, 40.5706754),(-73.3979416, 40.6113462),(-73.4617996, 40.8597849),
                    (-73.5170746, 40.9218144),(-73.6454773, 40.9057287),(-73.7491608, 40.8626411),(-73.7701035, 40.8099115),
                    (-73.7587738, 40.7908094),(-73.7476158, 40.7821012),(-73.7016964, 40.7520686),(-73.7005806, 40.7389983),
                    (-73.7088203, 40.7276165),(-73.7246990, 40.7243642),(-73.7285614, 40.7110932),(-73.7256432, 40.6850640),
                    (-73.7244415, 40.6519922),(-73.7386894, 40.6478245),(-73.7429810, 40.6394884),(-73.7488174, 40.6291970),
                    (-73.7682152, 40.6198161),(-73.7659836, 40.6153858),(-73.7546539, 40.6095217),(-73.7441397, 40.6120303),
                    (-73.7430668, 40.6082185),(-73.7384748, 40.6038200),(-73.7375307, 40.5968144),(-73.7380457, 40.5928388),
                    (-73.7421656, 40.5932951),(-73.7464571, 40.5924152),(-73.7534952, 40.5873638),(-73.7559843, 40.5847565),
                    (-73.7558985, 40.5801283)],
        
    'Jersey':      [(-74.0224457, 40.7328198),(-74.0452766, 40.7369822),(-74.0384102, 40.7501180),(-74.0437317, 40.7563598),
                    (-74.0519714, 40.7602607),(-74.0578079, 40.7685817),(-74.0760040, 40.7606508),(-74.0773773, 40.7510283),
                    (-74.0742874, 40.7464767),(-74.0780640, 40.7423149),(-74.0842438, 40.7388032),(-74.0911102, 40.7369822),
                    (-74.0955734, 40.7334702),(-74.1000366, 40.7192903),(-74.1105080, 40.7135655),(-74.1069031, 40.7021143),
                    (-74.1087914, 40.6984703),(-74.1029549, 40.6924834),(-74.0976334, 40.6846735),(-74.0990067, 40.6795966),
                    (-74.0921402, 40.6745193),(-74.0760040, 40.6681396),(-74.0657043, 40.6641031),(-74.0610695, 40.6694416),
                    (-74.0723991, 40.6746495),(-74.0648460, 40.6907913),(-74.0531731, 40.6881880),(-74.0459633, 40.6943055),
                    (-74.0354919, 40.6959975),(-74.0221024, 40.7324296)],
        
    'Bayonne':     [(-74.0669060, 40.6635822),(-74.0811539, 40.6686604),(-74.0942001, 40.6732173),(-74.0993500, 40.6798569),
                    (-74.0971184, 40.6838925),(-74.1012383, 40.6888389),(-74.1118813, 40.6926135),(-74.1350555, 40.6638427),
                    (-74.1465569, 40.6496479),(-74.1472435, 40.6437868),(-74.1360855, 40.6427448),(-74.1204643, 40.6441776),
                    (-74.1111946, 40.6469128),(-74.0964317, 40.6470431),(-74.0873337, 40.6506899),(-74.0744591, 40.6582433),
                    (-74.0677643, 40.6596757),(-74.0665627, 40.6626707),(-74.0669060, 40.6635822)],
        
    'WStaton':     [(-74.1228676, 40.6933945),(-74.1220093, 40.6957372),(-74.1680145, 40.7061485),(-74.2311859, 40.7035458),
                    (-74.3108368, 40.7175989),(-74.4049072, 40.6717852),(-74.5209503, 40.5618080),(-74.3918610, 40.4861263),
                    (-74.3480873, 40.4802509),(-74.3321228, 40.4896513),(-74.2971039, 40.5081872),(-74.2651749, 40.4957870),
                    (-74.2473221, 40.5260655),(-74.2449188, 40.5511133),(-74.2253494, 40.5577651),(-74.2071533, 40.5856691),
                    (-74.2038918, 40.6071760),(-74.2001152, 40.6347988),(-74.1831207, 40.6454801),(-74.1611481, 40.6436566),
                    (-74.1228676, 40.6933945)],
        
    'West':        [(-74.1144562, 40.6936548),(-74.1144562, 40.6939151),(-74.1058731, 40.7159075),(-74.0974617, 40.7287873),
                    (-74.0914536, 40.7372424),(-74.0806389, 40.7410793),(-74.0751457, 40.7453062),(-74.0766907, 40.7495978),
                    (-74.0904236, 40.7482974),(-74.0928268, 40.7542793),(-74.0914536, 40.7657215),(-74.0869904, 40.7836610),
                    (-74.0725708, 40.8026352),(-74.0488815, 40.8057537),(-74.0320587, 40.8132895),(-74.0368652, 40.8470604),
                    (-74.0608978, 40.8927534),(-74.0897369, 40.9316715),(-74.2112732, 41.0343051),(-74.2840576, 41.0317153),
                    (-74.3266296, 40.9907831),(-74.4207001, 40.9052097),(-74.4495392, 40.7992566),(-74.3431091, 40.7108330),
                    (-74.2613983, 40.6983402),(-74.1803741, 40.7066690),(-74.1144562, 40.6936548)],
        
    'WNewYork':    [(-74.0248489, 40.7325597),(-74.0181541, 40.7557097),(-74.0045929, 40.7752118),(-73.9857101, 40.8002962),
                    (-73.9678574, 40.8261505),(-73.9563560, 40.8536826),(-73.9699173, 40.8601744),(-73.9748955, 40.8721176),
                    (-73.9893150, 40.8784777),(-74.0076828, 40.8718580),(-74.0143776, 40.8671848),(-74.0336037, 40.8692618),
                    (-74.0336037, 40.8629007),(-74.0286255, 40.8556302),(-74.0293121, 40.8483589),(-74.0301704, 40.8405673),
                    (-74.0303421, 40.8338139),(-74.0344620, 40.8283586),(-74.0324020, 40.8199152),(-74.0327454, 40.8118604),
                    (-74.0378952, 40.8073129),(-74.0483665, 40.8065333),(-74.0631294, 40.8031550),(-74.0705109, 40.7961378),
                    (-74.0794373, 40.7875602),(-74.0868187, 40.7810614),(-74.0895653, 40.7774217),(-74.0875053, 40.7684517),
                    (-74.0900803, 40.7606508),(-74.0940285, 40.7541493),(-74.0933418, 40.7499880),(-74.0804672, 40.7512884),
                    (-74.0748024, 40.7484274),(-74.0744591, 40.7440056),(-74.0631294, 40.7395836),(-74.0445900, 40.7362018),
                    (-74.0250206, 40.7321695),(-74.0248489, 40.7325597)],
        
    'North':       [(-73.6434174, 40.9741957),(-73.6598969, 40.9866366),(-73.6523438, 41.0099570),(-73.7265015, 41.0984993),
                    (-73.4813690, 41.2117215),(-73.5500336, 41.2922536),(-73.5232544, 41.6051745),(-70.3399658, 41.6133889),
                    (-70.4333496, 41.2550973),(-71.6418457, 41.1041909),(-72.1142578, 41.2592268),(-72.9107666, 41.2096553),
                    (-73.4902954, 41.0109933),(-73.6406708, 40.9736773)],
    
    
    }
    
    x0 = row['pickup_longitude']
    y0 = row['pickup_latitude']
    x1 = row['dropoff_longitude']
    y1 = row['dropoff_latitude']
    
    # assing x0,y0 pair to a geographic location
    for key in locations:
        if Polygon(locations[key]).contains(Point(x0,y0)) == True:
            row['FROM'] = key
            break
            
    # if the point is not assigned then I will assign it to the closest object
    if row['FROM'] != row['FROM']:
        dist = {}
        for key in locations:
            dist[key] = Point(x0,y0).distance(Polygon(locations[key]))
            
        # finding the geo-location with minimum distance
        row['FROM'] = min(locations, key=locations.get)
        
    for key in locations:
        if Polygon(locations[key]).contains(Point(x1,y1)) == True:
            row['TO'] = key
            break
            
    if row['TO'] != row['TO']:
        dist = {}
        for key in locations:
            dist[key] = Point(x0,y0).distance(Polygon(locations[key]))
            
        # finding the geo-location with minimum distance
        row['TO'] = min(locations, key=locations.get)

    return pd.Series([row['FROM'], row['TO']])

def reorder_rename_columns(df, mode):
    cols = {
        'infer' : [
            'FROM',
            'TO',
            'hour',
            'weekday',
            'week',
            'year',
            'passenger_count',
            'dist_e',
            'dist_t',
            'pickup_longitude',
            'dropoff_longitude',
            'pickup_latitude',
            'dropoff_latitude'
        ],
        
        'train' : [
            'fare_amount',
            'FROM',
            'TO',
            'hour',
            'weekday',
            'week',
            'year',
            'passenger_count',
            'dist_e',
            'dist_t',
            'pickup_longitude',
            'dropoff_longitude',
            'pickup_latitude',
            'dropoff_latitude'
        ]
    }
    
    df = df[cols[mode]]
    
def add_extra_features(df):
    df.loc[:,'FROM'] = np.nan # pickup geo-location container
    df.loc[:,'TO'] = np.nan # dropoff geo-location container
    df.loc[:,'dist_e'] = np.nan # euclidean distance
    df.loc[:,'dist_t'] = np.nan # taxicab distance
    process_datetime(df) # add date-time features
    df[['FROM', 'TO']] = df.apply(geo_location, axis=1) # add geo-location
    
def distances(df):
    # euclidean distance
    df.loc[:, 'dist_e'] = np.sqrt(((df.loc[:,'x1'] - df.loc[:,'x0'])*50)**2 + ((df.loc[:,'y1'] - df.loc[:,'y0'])*69)**2)
    
    # manhattan distance
    df.loc[:, 'dist_t'] = abs(df.loc[:,'x1'] - df.loc[:,'x0'])*50 + abs(df.loc[:,'y1'] - df.loc[:,'y0'])*69

In [118]:
infer = pd.read_csv('../input/test.csv', index_col='key')

In [119]:
add_extra_features(infer)

In [120]:
infer = infer[cols['infer']]
infer.columns = new_cols['infer']
distances(infer)

In [197]:
infer.to_csv('../input/test_processed.csv')